In [1]:
import pandas as pd
import numpy as np 
import re
% matplotlib inline
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
import pickle
import time

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [4]:
clean_final = pd.read_csv("~/Documents/Capstone/code/clean_final_news.csv", encoding = "utf8", index_col = 0)

In [5]:
clean_final["binary_response"] = np.where(clean_final["response"] == "Not fake", 0, 1)

In [6]:
y_response = clean_final.binary_response.values

In [7]:
fileObject = open('trigram_text','rb')  
tri_lem_comb2 = pickle.load(fileObject)  

In [8]:
tfidf = TfidfVectorizer()
text_tfidf = tfidf.fit_transform(tri_lem_comb2)

In [17]:
#dense_text = text_tfidf.todense()

In [9]:
# Get list of all words in tfidf vocabulary
features = tfidf.get_feature_names()

In [12]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [13]:
def top_overall_words(text, response, features, cls=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in ids. '''
    if cls:
        ids = np.where(response == cls)
        D = text[ids].toarray()
    else:
        D = text.toarray()

    D[D < min_tfidf] = 0 # so common words across all documents won't dominate - may remove
    tfidf_mean = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_mean, features, top_n)

In [15]:
top_overall_words(text_tfidf, y_response, features)

MemoryError: 

In [ ]:
#def plot_tfidf_classfeats_h(dfs):
#    ''' Plot the data frames returned by the function plot_tfidf_classfeats(). '''
#    fig = plt.figure(figsize=(12, 9), facecolor="w")
#    x = np.arange(len(dfs[0]))
#    for i, df in enumerate(dfs):
#        ax = fig.add_subplot(1, len(dfs), i+1)
#        ax.spines["top"].set_visible(False)
#        ax.spines["right"].set_visible(False)
#        ax.set_frame_on(False)
#        ax.get_xaxis().tick_bottom()
#        ax.get_yaxis().tick_left()
#        ax.set_xlabel("Mean Tf-Idf Score", labelpad=16, fontsize=14)
#        ax.set_title("label = " + str(df.label), fontsize=16)
#        ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
#        ax.barh(x, df.tfidf, align='center', color='#3F5D7D')
#        ax.set_yticks(x)
#        ax.set_ylim([-1, x[-1]+1])
#        yticks = ax.set_yticklabels(df.feature)
#        plt.subplots_adjust(bottom=0.09, right=0.97, left=0.15, top=0.95, wspace=0.52)
#    plt.show()